<p align="left">
  <a href="https://colab.research.google.com/github/fernandoarcevega/AI_Workshop/blob/main/Part_1/05_PCA/05_PCA_mnl.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" width="200">
  </a>
</p>

In [ ]:
###############################################
# Author 1: Wilfrido Gómez-Flores (CINVESTAV) #
# Author 2: Fernando Arce-Vega (CIO)          #
# e-mail 1: wilfrido.gomez@cinvestav.mx       #
# e-mail 2: farce@cio.mx                      #
# Date:     nov/03/2025                       #
# Subject:  Principal component analysis      #
###############################################

In [ ]:
# Libraries
import numpy as np                       # Numerical array operations
import pandas as pd                      # Data manipulation/analysis
import matplotlib.pyplot as plt          # Data plotting/visualization

In [ ]:
# Other functions
!wget -q https://raw.githubusercontent.com/fernandoarcevega/AI_Workshop/main/helpers/misc.py
from misc import *

In [ ]:
# Load dataset
# Path to dataset
path = 'https://raw.githubusercontent.com/fernandoarcevega/AI_Workshop/main/datasets/bus_data.csv'
T = pd.read_csv(path)
data = T.values
n, d = data.shape
X = data[:, :d-1]  # Features
Y = data[:, d-1]   # Class labels

In [ ]:
# Check feature and targets shapes
print(f'Features shape: {X.shape}')
print(f'Targets shape:  {Y.shape}')

Features shape: (3287, 19)
Targets shape:  (3287,)


 🔵 🔵 🔵 🔵 🔵 🔵 🔵 🔵 🔵 🔵  (Total data)

 🟢 🟢 🟢 🟢 🟢 🟢 🟢 🟢 🟡 🟡  (Training and testing data)

In [ ]:
# Split dataset into training and test sets (80%-20%)
tr, tt = HoldOut(Y, 0.2)

# Training set
Xtr = X[tr, :]
Ytr = Y[tr]

# Test set
Xtt = X[tt, :]
Ytt = Y[tt]

In [ ]:
# Check feature and targets shapes for training and test sets
print(f'Training features shape: {Xtr.shape}')
print(f'Training targets shape:  {Ytr.shape} \n')
print(f'Testing features shape:  {Xtt.shape}')
print(f'Testing targets shape:   {Ytt.shape}')

Training features shape: (2630, 19)
Training targets shape:  (2630,) 

Testing features shape:  (657, 19)
Testing targets shape:   (657,)


In [ ]:
# Data normalization
Xtr, stats = zscorenorm(Xtr)
Xtt = zscorenorm(Xtt, stats)

### Algorithm: Principal Component Analysis (PCA) Steps

1.  Centralize data to have a zero mean
2.  Compute the covariance matrix
3.  Obtain eigenvalues and eigenvectors
4.  Sort eigenvalues in descending order
5.  Reorder eigenvectors accordingly
6.  Pick the first $q$ sorted eigenvectors
7.  Project the original data onto the selected eigenvectors (i.e., the largest PCS)


In [ ]:
# PCA
def my_pca(X):
    n = X.shape[0]                                        # Number of samples
    mn = np.mean(X, axis=0)                               # Mean of the columns
    X_centered = X - mn                                   # Centralize data to have a zero mean
    S = np.dot(X_centered.T, X_centered) / n              # Compute the covariance matrix
    eigenvalues, eigenvectors = np.linalg.eig(S)          # Obtain eigenvalues and eigenvectors
    eigenvectors = np.real_if_close(eigenvectors, tol=1)  # Real components
    eigenvalues = np.real_if_close(eigenvalues, tol=1)    # Real components
    idx = eigenvalues.argsort()[::-1]                     # Sort eigenvalues in descending order
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]                   # Reorder eigenvectors accordingly
    R = 1 - eigenvalues / np.sum(eigenvalues)             # Explained variance
    q = np.argmax(R > 0.95) + 1                           # Top q PCs
    W = eigenvectors[:, :q]                               # Pick the first q sorted eigenvectors
    return W, q

In [ ]:
# Main logistic regression
def LRtrain(X, Y, eta, tmax):
    c = np.unique(Y)
    nc = len(c)
    n = X.shape[0]
    X = np.hstack((np.ones((n, 1)), X))     # Augments for bias

    if nc > 2:
        T = np.zeros((n, nc))

        for i in range(nc):
            T[Y == c[i], i] = 1             # One-hot encoding

        W, L = trainMclass(X, T, eta, tmax) # Softmax regression

    else:
        W, L = train2class(X, Y, eta, tmax) # Logistic regression

    return W, L

In [ ]:
# Train two-class logistic regression
def train2class(X, T, eta, tmax):
    d = X.shape[1]                    # Number of features
    W = np.zeros(d)                   # Weight vector
    loss = np.zeros(tmax)

    for t in range(tmax):             # Gradient descent loop
        P = 1 / (1 + np.exp(-X @ W))  # Sigmoid function
        W -= eta * (P - T) @ X        # Update weights

        # Binary cross-entropy loss
        loss[t] = -np.mean(T * np.log(P) + (1 - T) * np.log(1 - P))

    return W, loss

In [ ]:
# Train multiclass logistic regression
def trainMclass(X, T, eta, tmax):
    d = X.shape[1]        # Number of features
    c = T.shape[1]        # Number of classes
    W = np.zeros((c, d))  # Weight matrix
    loss = np.zeros(tmax)

    # Gradient descent loop
    for t in range(tmax):
        E = np.exp(X @ W.T)
        P = E / np.sum(E, axis=1, keepdims=True)  # Softmax
        W -= eta * (P - T).T @ X                  # Update weights

        # Categorical cross-entropy loss
        loss[t] = -np.mean(T * np.log(P))
    return W, loss

In [ ]:
# Predict with the trained linear model
def LRpredict(X, W):
    n = X.shape[0]
    X = np.hstack((np.ones((n, 1)), X))   # Augmented

    if W.ndim > 1:
        E = np.exp(X @ W.T)
        Pr = E / np.sum(E, axis=1, keepdims=True) # Softmax
        Ypp = np.argmax(Pr, axis=1)               # Classify
        Ypp = Ypp+1

    else:
        Pr = 1 / (1 + np.exp(-X @ W))   # Sigmoid
        Ypp = (Pr > 0.5).astype(float)  # Classify

    return Ypp, Pr

In [ ]:
# PCA
Wpca, q = my_pca(Xtr)
Xtr2 = np.dot(Xtr, Wpca)
Xtt2 = np.dot(Xtt, Wpca)

In [ ]:
# Principal components
print(f'Number of principal compoents in 95% of variance: {np.shape(Wpca)}')

Number of principal compoents in 95% of variance: (19, 7)


In [ ]:
# Without PCA
# Train model
eta = 1e-4
tmax = 1000
W, _ = LRtrain(Xtr, Ytr, eta, tmax)

In [ ]:
# Classify test data
Ypp, _ = LRpredict(Xtt, W)
err = np.mean(Ypp != Ytt)
print(f'Error without PCA: {100 * err:.3f}%')

Error without PCA: 15.677%


In [ ]:
# With PCA
# Train model
W, _ = LRtrain(Xtr2, Ytr, eta, tmax)

# Classify test data
Ypp, _ = LRpredict(Xtt2, W)
err = np.mean(Ypp != Ytt)
print(f'Error with {q} PCs: {100 * err:.3f}%')

Error with 7 PCs: 15.982%
